In [1]:
#Install some required packages

!pip install --quiet "pytorch-lightning" "torchmetrics" "torchvision" "torch"
!pip install --quiet ipywidgets widgetsnbextension pandas-profiling
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from ipywidgets import FloatProgress
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

####################################################################################
####################################################################################
# Set version of dataset to train CNN model on
#version = 'cdetect'
#version = 'cdetect_v2'
#version = 'cdetect_compose'
#version = 'cdetect_all'
version = 'cdetect_v2_with_gan'
####################################################################################
####################################################################################

# Store tensorboard logs
writer = SummaryWriter(log_dir="/workspace/data/"+version)

In [3]:
#New Train/Val/Test Split (v2) (baseline image set)
!wget -O cdetect_v2.zip "https://www.dropbox.com/s/gk6yr3anobz5nkn/cdetect_v2.zip?dl=0"
!unzip -q cdetect_v2.zip -d /workspace/cdetect_v2_with_gan

--2022-04-10 03:29:31--  https://www.dropbox.com/s/gk6yr3anobz5nkn/cdetect_v2.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/gk6yr3anobz5nkn/cdetect_v2.zip [following]
--2022-04-10 03:29:31--  https://www.dropbox.com/s/raw/gk6yr3anobz5nkn/cdetect_v2.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5101d20a6675bbaea619057ff0.dl.dropboxusercontent.com/cd/0/inline/BjEE4xsQGAExTIEZ5VRWip0-h_VSLIU3qiMUSlI2fzq0Ajg4P6uTGsj073S_WkWUWxncgkRj9PwGt03olRmsIqOHQVblhlFpLlZmeQwK-mae-48oPecqKkunELXb3dax5iJOaFAgSJ3AURROHJQGhxPqIzYgsq8dQJc7zj4cyL4gpA/file# [following]
--2022-04-10 03:29:32--  https://uc5101d20a6675bbaea619057ff0.dl.dropboxusercontent.com/cd/0/inline/BjEE4xsQGAExTIEZ5VRWip0-h_VSLIU3qiMUSlI2fzq0Ajg4P6uTGsj07

In [4]:
#Add GAN images to baseline image set
!wget -O gan_images.zip "https://www.dropbox.com/s/djye48jsempvtok/gan_images.zip?dl=0"
!unzip -q gan_images.zip -d /workspace/cdetect_v2_with_gan/train

--2022-04-10 03:31:45--  https://www.dropbox.com/s/djye48jsempvtok/gan_images.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/djye48jsempvtok/gan_images.zip [following]
--2022-04-10 03:31:45--  https://www.dropbox.com/s/raw/djye48jsempvtok/gan_images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc76446a68f0b54f4c1fe3210e00.dl.dropboxusercontent.com/cd/0/inline/BjHxjLkvYx1mt5wIq3-ZTNilmJPJAASUxG7Dw4cwhFUy9_m8hFPpqVR0kiUo0xU_U9eBB1k9TBLnaDeCutfFeJL5FJGWoOwE7L7sOeLqFeQigspTlsdxk64bddmLT__XIDVjsrWPAnXQ25NzxBNY8dRhMG07KEUTdePjuM3gVapXsQ/file# [following]
--2022-04-10 03:31:45--  https://uc76446a68f0b54f4c1fe3210e00.dl.dropboxusercontent.com/cd/0/inline/BjHxjLkvYx1mt5wIq3-ZTNilmJPJAASUxG7Dw4cwhFUy9_m8hFPpqVR0k

In [39]:
# Moved the extracted GAN training images into their correct directories for COVID/bacterial/normal

# Convert GAN .png images to .jpg, then remove the .png files

from PIL import Image

classes = ['normal', 'covid', 'bacterial']

for cls in classes:
    directory = '/workspace/cdetect_v2_with_gan/train/{}/'.format(cls)

    for filename in os.listdir(directory):
        name = (''.join(os.path.splitext(filename)))
        if name.find('.png') > 0:
            end = name.find('.png')
            im1 = Image.open(directory+name)
            rgb_im = im1.convert('RGB')
            rgb_im.save(directory+name[:end]+'.jpg')
            
    # Remove .png images
    for file in os.listdir(directory):
        if file.endswith('.png'):
                os.remove(directory+file) 

In [3]:
SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True
torch.cuda.device_count()

ARCH = 'vgg16'
START_EPOCH = 0
EPOCHS = 100
LR = 0.0001 #LOWERED LR BY A FACTOR OF 10 FOR PRETRAINED MODEL
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 1
TRAIN_BATCH=16
VAL_BATCH=16
TEST_BATCH=16
WORKERS=2
TRAINDIR="/workspace/"+version+"/train"
VALDIR="/workspace/"+version+"/val"
TESTDIR="/workspace/"+version+"/test"

if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

GPU = 0
torch.cuda.set_device(GPU)
cudnn.benchmark = True

In [4]:
def train(train_loader, model, criterion, optimizer):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    #top3 = AverageMeter('Acc@3', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1],
        #[batch_time, data_time, losses, top1],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target, path) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc3 = accuracy(output, target, topk=(1, 3))
        #acc1 = accuracy(output, target, topk=(1, 1))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        #top3.update(acc3[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)
            
            
def validate(val_loader, model, criterion, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target, path) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc3 = accuracy(output, target, topk=(1, 3))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        print(' * Acc@1 {top1.avg:.3f}'
              .format(top1=top1))

    return top1.avg

def save_checkpoint(state, is_best, filename='/workspace/'+version+'/checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, '/workspace/'+version+'/model_best.pth.tar')
        
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
    
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [5]:
#Need to be able to access file path for our train/val/test dataloader so we can output CNN model predictions with file path
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. 
    Extends torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [6]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
# cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
# cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
# cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
# cifar_std_RGB = [0.2023, 0.1994, 0.2010]

IMG_SIZE = 512
NUM_CLASSES = 3

In [7]:
#Try pretrained vs random weight initializations to see how we do!
#Pre-Trained
model = models.__dict__[ARCH](pretrained=True)
#Random
#model = models.__dict__[ARCH]()

# Visualize architecture
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
# Had to go into /workspace/cdetect_v2_with_gan and < + /train to remove checkpoints file
#rm -r .ipynb_checkpoints/

In [15]:
#model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

#We want to change the model's output layer to 3 classes (covid/normal/bacterial)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, NUM_CLASSES)
model.cuda(GPU)

criterion = nn.CrossEntropyLoss().cuda(GPU)

optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

transform_val = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])


train_dataset = ImageFolderWithPaths(root=TRAINDIR, transform=transform_train)

val_dataset = ImageFolderWithPaths(root=VALDIR, transform=transform_val)

# Can use transform_val for the test dataset as well
test_dataset = ImageFolderWithPaths(root=TESTDIR, transform=transform_val)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
        num_workers=WORKERS, pin_memory=True, sampler=None)

val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=TEST_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

best_acc1 = 0

In [46]:
# log_dir = "/workspace/data/"+version
# %load_ext tensorboard
# %tensorboard --logdir /workspace/data/cdetect_v2

# Train Model & Save Checkpoints

In [16]:
for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, epoch)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][ 0/10]	Time 11.038 (11.038)	Data  0.364 ( 0.364)	Loss 1.2123e+00 (1.2123e+00)	Acc@1  18.75 ( 18.75)
Epoch: [0][ 1/10]	Time  0.383 ( 5.710)	Data  0.038 ( 0.201)	Loss 9.7117e-01 (1.0917e+00)	Acc@1  62.50 ( 40.62)
Epoch: [0][ 2/10]	Time  1.012 ( 4.144)	Data  0.668 ( 0.357)	Loss 1.1111e+00 (1.0982e+00)	Acc@1  37.50 ( 39.58)
Epoch: [0][ 3/10]	Time  1.013 ( 3.361)	Data  0.672 ( 0.436)	Loss 1.0152e+00 (1.0774e+00)	Acc@1  56.25 ( 43.75)
Epoch: [0][ 4/10]	Time  1.005 ( 2.890)	Data  0.666 ( 0.482)	Loss 1.0224e+00 (1.0664e+00)	Acc@1  50.00 ( 45.00)
Epoch: [0][ 5/10]	Time  1.010 ( 2.577)	Data  0.667 ( 0.513)	Loss 1.1196e+00 (1.0753e+00)	Acc@1  31.25 ( 42.71)
Epoch: [0][ 6/10]	Time  1.015 ( 2.354)	Data  0.673 ( 0.536)	Loss 1.0547e+00 (1.0724e+00)	Acc@1  43.75 ( 42.86)
Epoch: [0][ 7/10]	Time  1.016 ( 2.187)	Data  0.674 ( 0.553)	Loss 1.0835e+00 (1.0737e+00)	Acc@1  25.00 ( 40.62)
Epoch: [0][ 8/10]	Time  1.015 ( 2.056)	Data  0.674 ( 0.566)	Loss 1.1690e+00 (1.0843e+00)	Acc@1  37.50 ( 40.28)
E